In [153]:
import requests
import pandas as pd
from pandas import json_normalize
import random

merged_df = pd.DataFrame()
medianums= ['190', '680', '742', '888', '1']

#for num in range(1,10):
for num in medianums:
    url = f'https://mediadive.dsmz.de/rest/medium-composition/{num}'
    if requests.get(url).status_code==200: 
        df = json_normalize(requests.get(url).json().get('data'))[["name", "g_l"]]
        df = df.rename(columns={"g_l": f"{num}"})  # give each column a unique name
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on="name", how="outer")
    else:
        print('error',requests.get(url).status_code, ' for ',num)

medianums = list(merged_df.columns[1:])
print('medianums available:', medianums)
merged_df.head()

medianums available: ['190', '680', '742', '888', '1']


,name,190,680,742,888,1
0,(DL)-alpha-Lipoic acid,NaN,NaN,0.000049,NaN,NaN
1,(NH4)2SO4,NaN,NaN,NaN,1.000000,NaN
2,Agar,15.0,20.0,NaN,0.500000,15.0
3,AlK(SO4)2 x 12 H2O,NaN,NaN,0.000198,NaN,NaN
4,B12,NaN,NaN,NaN,0.000001,NaN


In [154]:
df_list=[]
for num in medianums:
    url = f'https://mediadive.dsmz.de/rest/medium-strains/{num}'
    if requests.get(url).status_code==200: 
        df = json_normalize(requests.get(url).json().get('data'))[["species","bacdive_id"]]
        df = df.rename(columns={"species": f"species_{num}","bacdive_id": f"id_{num}"})  # give each column a unique name
        df_list.append(df)
    else:
        print('error',requests.get(url).status_code, ' for ',num)
merged_dfstrains = pd.concat(df_list, axis=1)

merged_dfstrains.head()

bacdive_id_dict = {}

for num in medianums:
    url = f'https://mediadive.dsmz.de/rest/medium-strains/{num}'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json().get('data', [])
        if data:  # check if data list is not empty
            df = json_normalize(data)
            if "bacdive_id" in df.columns and not df.empty:
                first_id = df.iloc[0]["bacdive_id"]  # iloc needs integer index and column name as string
                bacdive_id_dict[num] = first_id
            else:
                print(f"No 'bacdive_id' found in data for {num}")
                bacdive_id_dict[num] = None
        else:
            print(f"No data found for {num}")
            bacdive_id_dict[num] = None
    else:
        print(f"Error {response.status_code} for {num}")
        bacdive_id_dict[num] = None

bacdive_id_dict


{'190': None, '680': 11053, '742': 4034, '888': 10442, '1': 654.0}

In [155]:
import bacdive
client = bacdive.BacdiveClient('tanliangxun2000@gmail.com', 'pass1234')

taxid_medianum_dict = {}
for num in medianums:
    bacdive_id = bacdive_id_dict.get(num)
    print(f"medianum: {num} Strain bacdiveID: {bacdive_id}")
    
    if bacdive_id is not None:
        strain_data = client.search(id=int(bacdive_id))
        
        for strain in client.retrieve(['NCBI tax id']):
            strain_key = list(strain)[0]
            ncbi_list = strain[strain_key]  # this is always a list
            
            for entry in ncbi_list:
                ncbi_info = entry.get('NCBI tax id')
                
                if isinstance(ncbi_info, dict):
                    tax_id = ncbi_info.get('NCBI tax id')
                    level = ncbi_info.get('Matching level')
                    if level == 'species':
                        print(f"Tax ID: {tax_id}")

                elif isinstance(ncbi_info, list):
                    for subentry in ncbi_info:
                        tax_id = subentry.get('NCBI tax id')
                        level = subentry.get('Matching level')
                        if level == 'species':
                            print(f"Tax ID: {tax_id}")

                else:
                    # You can skip or print unexpected formats if needed
                    pass
        taxid_medianum_dict[tax_id]= num
    else:
        print(f"No valid BacDive ID found for {num}")

taxid_medianum_dict
# client.search(id='4437;4453')

# # for strain in client.retrieve():
# #     print(strain["General"])
# for strain in client.retrieve(['NCBI tax id']):
#     # print(strain)
#     # print(strain[list(strain)[0]])
#     print(strain[list(strain)[0]][0]['NCBI tax id']['NCBI tax id'])

-- Authentication successful --
medianum: 190 Strain bacdiveID: None
No valid BacDive ID found for 190
medianum: 680 Strain bacdiveID: 11053
Tax ID: 39122
medianum: 742 Strain bacdiveID: 4034
Tax ID: 53245
medianum: 888 Strain bacdiveID: 10442
Tax ID: 682713
medianum: 1 Strain bacdiveID: 654.0
Tax ID: 1398


{39122: '680', 53245: '742', 682713: '888', 1398: '1'}

In [83]:
import subprocess
import shlex   # for safe quoting if you ever need it

# Grab the *values* (index 1) from each (key, value) pair
taxid_list = [item[1] for item in media_ncbi_dict.items()]   # e.g. [1, 3, 4]

# Build the command as a list (avoid shell-string pitfalls)
cmd = [
    "/Users/liangxuntan/Code/fyp2025/scripts/run_taxid2fna.sh",
    *map(str, [562, 384])            # unpack TaxIDs as separate args
]

# Run it
subprocess.run(cmd, check=True)

→ Processing TaxID 562
→ Processing TaxID 384
✓ Finished. Full log: /Users/liangxuntan/Code/fyp2025/data/logs/taxid2fna_20250519_105046.log


CompletedProcess(args=['/Users/liangxuntan/Code/fyp2025/scripts/run_taxid2fna.sh', '562', '384'], returncode=0)

In [99]:
import subprocess
from pathlib import Path

fasta_dir = Path("/Users/liangxuntan/Code/fyp2025/data/labelled_genomes")
fna_files = sorted(fasta_dir.glob("*.fna"))

# --- or: run on ALL .fna files ---------------------------------------------
for fna in fna_files:
#for fna in [fna_files[0]]:
    subprocess.run(
        ["/Users/liangxuntan/Code/fyp2025/scripts/fna2prot.py", str(fna)],
        check=True
    )

/Users/liangxuntan/anaconda3/envs/mne/lib/python3.11/site-packages/Bio/SeqRecord.py:228: BiopythonDeprecationWarning: Using a string as the sequence is deprecated and will raise a TypeError in future. It has been converted to a Seq object.
  warnings.warn(


Written 350 protein sequences to '/Users/liangxuntan/Code/fyp2025/data/prodigal_output/222_predictedproteins.prot'


In [102]:
from pathlib import Path
from fna2prot_parallel import run_all

fasta_dir = Path("/Users/liangxuntan/Code/fyp2025/data/labelled_genomes")
fna_files = sorted(fasta_dir.glob("*.fna"))
evalue = "1e-50"

results = run_all(fna_files, evalue, nproc=4)

print("Protein prediction output files:")
for r in results:
    print(r)

/Users/liangxuntan/anaconda3/envs/mne/lib/python3.11/site-packages/Bio/SeqRecord.py:228: BiopythonDeprecationWarning: Using a string as the sequence is deprecated and will raise a TypeError in future. It has been converted to a Seq object.
  warnings.warn(
/Users/liangxuntan/anaconda3/envs/mne/lib/python3.11/site-packages/Bio/SeqRecord.py:228: BiopythonDeprecationWarning: Using a string as the sequence is deprecated and will raise a TypeError in future. It has been converted to a Seq object.
  warnings.warn(
/Users/liangxuntan/anaconda3/envs/mne/lib/python3.11/site-packages/Bio/SeqRecord.py:228: BiopythonDeprecationWarning: Using a string as the sequence is deprecated and will raise a TypeError in future. It has been converted to a Seq object.
  warnings.warn(


Written 350 protein sequences to '/Users/liangxuntan/Code/fyp2025/data/prodigal_output/222_predictedproteins.prot'


/Users/liangxuntan/anaconda3/envs/mne/lib/python3.11/site-packages/Bio/SeqRecord.py:228: BiopythonDeprecationWarning: Using a string as the sequence is deprecated and will raise a TypeError in future. It has been converted to a Seq object.
  warnings.warn(


Written 210 protein sequences to '/Users/liangxuntan/Code/fyp2025/data/prodigal_output/562_predictedproteins.prot'
Written 346 protein sequences to '/Users/liangxuntan/Code/fyp2025/data/prodigal_output/382_predictedproteins.prot'
Written 321 protein sequences to '/Users/liangxuntan/Code/fyp2025/data/prodigal_output/384_predictedproteins.prot'
Protein prediction output files:
/Users/liangxuntan/Code/fyp2025/data/prodigal_output/222_predictedproteins.prot
/Users/liangxuntan/Code/fyp2025/data/prodigal_output/382_predictedproteins.prot
/Users/liangxuntan/Code/fyp2025/data/prodigal_output/384_predictedproteins.prot
/Users/liangxuntan/Code/fyp2025/data/prodigal_output/562_predictedproteins.prot


In [108]:
from pathlib import Path
import subprocess

prodigal_dir = Path("/Users/liangxuntan/Code/fyp2025/data/prodigal_output")
prodigal_files = sorted(prodigal_dir.glob("*.prot"))
pfamscript = '/Users/liangxuntan/Code/fyp2025/scripts/prot2pfam.sh'

# Convert Path objects to strings and join with spaces
file_args = " ".join(str(f) for f in prodigal_files)

# Full command as a string
cmd = f"{pfamscript} {file_args}"

# Run the command using shell=True
result = subprocess.run(cmd, shell=True, capture_output=True, text=True)

# Print stdout and stderr for debugging
print(result.stdout)
print(result.stderr)

▶ Starting parallel hmmscan ( Mon May 19 13:32:29 BST 2025 ) on 4 file(s)
Running 10 job(s) × 1 thread(s) each
→ Processing 562_predictedproteins.prot ➜ /Users/liangxuntan/Code/fyp2025/data/hmmer_output/562_pfam.tbl
✓ Finished 562_predictedproteins.prot in 64s
→ Processing 384_predictedproteins.prot ➜ /Users/liangxuntan/Code/fyp2025/data/hmmer_output/384_pfam.tbl
✓ Finished 384_predictedproteins.prot in 91s
→ Processing 382_predictedproteins.prot ➜ /Users/liangxuntan/Code/fyp2025/data/hmmer_output/382_pfam.tbl
✓ Finished 382_predictedproteins.prot in 96s
→ Processing 222_predictedproteins.prot ➜ /Users/liangxuntan/Code/fyp2025/data/hmmer_output/222_pfam.tbl
✓ Finished 222_predictedproteins.prot in 97s
✔ All done ( Mon May 19 13:34:06 BST 2025 ). Full log: /Users/liangxuntan/Code/fyp2025/data/logs/prot2pfam_20250519_133229.log




In [156]:
from pathlib import Path
import collections
import pandas as pd

hmmer_dir = Path("/Users/liangxuntan/Code/fyp2025/data/hmmer_output")
hmmer_files = sorted(hmmer_dir.glob("*.tbl"))

count_dict = {}

for f in hmmer_files:
    pfam_counts = collections.Counter()

    with f.open() as fh:                      # <— use f, not pfam_tbl_path
        for line in fh:
            if line.startswith("#") or not line.strip():
                continue
            fields = line.split()
            pfam_name = fields[0]             # 1st column = Pfam HMM name
            pfam_counts[pfam_name] += 1

    taxid = f.stem.split('_')[0]              # filename: <taxid>_pfam.tbl
    count_dict[taxid] = pfam_counts

# convert nested Counter dict to DataFrame (taxid as rows, Pfam as columns)
pfam_df = pd.DataFrame.from_dict(count_dict, orient="index").fillna(0).astype(int)


print(pfam_df.head()) 
#pfam_df.to_csv("pfam_matrix.tsv", sep='\t')


     DotU  LysR_substrate  HTH_1  Relaxase  RepB_primase  RepB_primase_C  \
222     1              39     10         1             1               1   
384     0               1      0         0             0               0   
382     0               0      0         0             0               0   
562     0               0      0         0             0               0   

     ADH_N  ADH_zinc_N  TctC  ABC_tran  ...  FBPase_glpX  tRNA_U5-meth_tr  \
222      2           3    13        28  ...            0                0   
384      2           2     1        11  ...            0                0   
382      2           2     1        14  ...            0                0   
562      2           3     0         4  ...            2                2   

     PhnI  Phosphonate-bd  PTR2  GTP-bdg_N  GTP-bdg_M  MMR_HSR1  AidB_N  CreD  
222     0               0     0          0          0         0       0     0  
384     0               0     0          0          0         0       0  

In [169]:
import pandas as pd
from collections import defaultdict

taxid2media_dict={222: '680', 384: '742', 382: '888', 562: '1'}

# normalise mapping: int keys, values always list
media_map = {
    str(k): (v if isinstance(v, list) else [v])
    for k, v in taxid2media_dict.items()
}

used_counter = defaultdict(int)
missing_taxids = []          # ← collect unknown IDs here

def pick_media_id(taxid):
    if taxid not in media_map:
        missing_taxids.append(taxid)   # record once per appearance
        return taxid                   # keep original label
    i = used_counter[taxid]
    id_list = media_map[taxid]
    used_counter[taxid] += 1
    return id_list[i] if i < len(id_list) else id_list[-1]

# rebuild index
pfam_df.index = [pick_media_id(tid) for tid in pfam_df.index]

# Convert DataFrame index to int if possible
pfam_df.index = pfam_df.index.astype(int)


if missing_taxids:
    print("TaxIDs without media ID:", sorted(set(str(t) for t in missing_taxids)))

pfam_df.head()

TaxIDs without media ID: ['1', '680', '742', '888']


,DotU,LysR_substrate,HTH_1,Relaxase,RepB_primase,RepB_primase_C,ADH_N,ADH_zinc_N,TctC,ABC_tran,...,FBPase_glpX,tRNA_U5-meth_tr,PhnI,Phosphonate-bd,PTR2,GTP-bdg_N,GTP-bdg_M,MMR_HSR1,AidB_N,CreD
680,1,39,10,1,1,1,2,3,13,28,...,0,0,0,0,0,0,0,0,0,0
742,0,1,0,0,0,0,2,2,1,11,...,0,0,0,0,0,0,0,0,0,0
888,0,0,0,0,0,0,2,2,1,14,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,2,3,0,4,...,2,2,2,1,2,1,1,1,1,2


In [ ]:

import requests
from pandas import json_normalize
import pandas as pd

medianums = list(media_df.index)
merged_df = pd.DataFrame()

with requests.Session() as s:                       # ← one session for all calls
    for num in medianums:
        url = f'https://mediadive.dsmz.de/rest/medium-composition/{num}'
        resp = s.get(url)                           # use session

        if resp.status_code == 200:
            data = resp.json().get('data')
            if data:
                idf = json_normalize(data)[["name", "g_l"]]
                idf = idf.rename(columns={"g_l": str(num)})
                merged_df = (
                    idf if merged_df.empty
                    else pd.merge(merged_df, idf, on="name", how="outer")
                )
        else:
            print(f"Error {resp.status_code} for medium ID {num}")
            
# names you want on the columns and the index
rownames   = merged_df["name"].values          # example source
medianums  = merged_df.columns[1:]             # example source (skip "name" col)

media_df = merged_df.set_index("name").T       # 1) make "name" the index, 2) transpose
media_df.index = medianums                     # index (rows) = medium IDs
media_df.columns = rownames                    # columns     = ingredient names

media_df.index =  media_df.index.astype(int)
media_df = (media_df > 0).astype(int)
media_df.head()


,(DL)-alpha-Lipoic acid,(NH4)2SO4,Agar,AlK(SO4)2 x 12 H2O,B12,Biotin,CaCl2 x 2 H2O,CaCl2 x 6 H2O,Calcium D-(+)-pantothenate,Calcium pantothenate,...,Riboflavin,Sodium resazurin,Sodium succinate,Thiamine,Thiamine HCl,Tryptone,Vitamin B12,Yeast extract,ZnSO4 x 7 H2O,p-Aminobenzoic acid
680,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
742,1,0,0,1,0,1,1,0,1,0,...,1,1,0,0,1,0,1,1,1,1
888,0,1,1,0,1,1,0,1,0,1,...,1,0,1,1,0,0,0,0,1,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [172]:
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

# --- predictors -------------------------------------------------------------
# pfam_df: rows = genomes (taxid), cols = Pfam counts   (built earlier)
# media_df: rows = same taxid index, cols = binary media traits
X = pfam_df.join(media_df, how="inner", lsuffix="_pfam")
y = X[media_df.columns]
X = X[pfam_df.columns]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=None)  # stratify if not too sparse

# --- model ------------------------------------------------------------------
base_rf = RandomForestClassifier(
            n_estimators=500,
            max_depth=None,
            n_jobs=-1,
            class_weight="balanced_subsample",  # handles label imbalance
            random_state=42)

clf = MultiOutputClassifier(base_rf)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

# --- evaluation -------------------------------------------------------------
for i, label in enumerate(y.columns):
    print(f"\nLabel: {label}")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i], digits=3))
    
# Optional overall metrics
exact_match = accuracy_score(y_test, y_pred)      # all labels correct
macro_f1 = f1_score(y_test, y_pred, average="macro")
print(f"\nExact-match accuracy: {exact_match:.3f}")
print(f"Macro-averaged F1    : {macro_f1:.3f}")


Label: (DL)-alpha-Lipoic acid
              precision    recall  f1-score   support

           0      0.000     0.000     0.000       0.0
           1      0.000     0.000     0.000       1.0

    accuracy                          0.000       1.0
   macro avg      0.000     0.000     0.000       1.0
weighted avg      0.000     0.000     0.000       1.0


Label: (NH4)2SO4
              precision    recall  f1-score   support

           0      1.000     1.000     1.000         1

    accuracy                          1.000         1
   macro avg      1.000     1.000     1.000         1
weighted avg      1.000     1.000     1.000         1


Label: Agar
              precision    recall  f1-score   support

           0      0.000     0.000     0.000       1.0
           1      0.000     0.000     0.000       0.0

    accuracy                          0.000       1.0
   macro avg      0.000     0.000     0.000       1.0
weighted avg      0.000     0.000     0.000       1.0


Label: AlK

/Users/liangxuntan/anaconda3/envs/mne/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/liangxuntan/anaconda3/envs/mne/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/liangxuntan/anaconda3/envs/mne/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitali